In [3]:
import pandas as pd
import numpy as np

def load_data(option):
    if option == 0:
        with open('sample.txt') as f:
            data = f.read().splitlines()
    else:
        with open('input.txt') as f:
            data = f.read().splitlines()

    arr = pd.DataFrame(data).to_numpy()
    patterns = [pat.strip() for pat in arr[0][0].split(',')]
    designs = [des[0] for des in arr[2:]]

    return patterns, designs

patterns, designs = load_data(0)
print(patterns, designs)

['r', 'wr', 'b', 'g', 'bwu', 'rb', 'gb', 'br'] ['brwrr', 'bggr', 'gbbr', 'rrbgbr', 'ubwu', 'bwurrg', 'brgr', 'bbrgwb']


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import heapq

patterns, designs = load_data(1)

# Gather a dictionary of patterns for fast access
patternDict = {}
for pattern in patterns:
    firstChar = pattern[0]
    if firstChar not in patternDict:
        patternDict[firstChar] = [pattern]
    else:
        patternDict[firstChar].append(pattern)

total = 0
for design in designs:
    minHeap = [(len(design), design)] # remaining chars, remaining design to match
    found = False

    while minHeap and not found:
        remainChars, subdesign = heapq.heappop(minHeap)
        firstChar = subdesign[0]

        if firstChar in patternDict:
            for pattern in patternDict[firstChar]:
                match = re.search(pattern, subdesign)

                if match != None and match.start() == 0:
                    if len(pattern) == remainChars: # Found possible designs
                        found = True
                        total += 1
                    else:
                        heapq.heappush(minHeap, (remainChars - len(pattern), subdesign[len(pattern):]))

print(total)

213


In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

patterns, designs = load_data(1)

# Gather a dictionary of patterns for fast access
patternDict = {}
for pattern in patterns:
    firstChar = pattern[0]
    if firstChar not in patternDict:
        patternDict[firstChar] = [pattern]
    else:
        patternDict[firstChar].append(pattern)

# Memoization
memo = {}

# Recursive loop
def recursive(remainChars, subdesign):
    if subdesign in memo:
        return memo[subdesign]
    
    firstChar = subdesign[0]
    subtotal = 0
    if firstChar in patternDict:
        for pattern in patternDict[firstChar]:
            match = re.search(pattern, subdesign)

            if match != None and match.start() == 0:
                if len(pattern) == remainChars: # Base case
                    subtotal += 1
                else:
                    subtotal += recursive(remainChars - len(pattern), subdesign[len(pattern):])
        
    if subdesign not in memo:
        memo[subdesign] = subtotal
    return subtotal

# Check each design
total = 0
for design in designs:
    total += recursive(len(design), design)

print(total)

1016700771200474


In [62]:
from typing import List
import math

def mergeSort(position: List[int], speed: List[int]):
    s, e = 0, len(position)

    # Base case
    if len(position) == 1:
        return position, speed
    
    m = math.ceil((s + e) / 2)
    first_pos, first_spd = mergeSort(position[0:m], speed[0:m])
    second_pos, second_spd = mergeSort(position[m:], speed[m:])

    # Merge
    sorted_pos = []
    sorted_spd = []
    while len(sorted_pos) < len(position):
        # Default case:
        if len(first_pos) == 0:
            sorted_pos.extend(second_pos)
            sorted_spd.extend(second_spd)
        elif len(second_pos) == 0:
            sorted_pos.extend(first_pos)
            sorted_spd.extend(first_spd)
        else:
            if first_pos[0] <= second_pos[0]:
                sorted_pos.append(first_pos.pop(0))
                sorted_spd.append(first_spd.pop(0))
            else:
                sorted_pos.append(second_pos.pop(0))
                sorted_spd.append(second_spd.pop(0))

    return sorted_pos, sorted_spd

# Sort the position list
sorted_pos, sorted_spd = mergeSort([1,15,6,8,18,14,16,2,19,17,3,20,5], [8,5,5,7,10,10,7,9,3,4,4,10,2])
target = 21

# Count the number of possible car fleet
# At first, each car is its own fleet
total_fleet = len(sorted_pos)
i = len(sorted_pos)-2
while i >= 0:
    t_1 = (target - sorted_pos[i]) / sorted_spd[i]
    t_2 = (target - sorted_pos[i+1]) / sorted_spd[i+1]

    # Can car 1 catch up with car 2?
    if t_1 <= t_2:
        sorted_pos.pop(i)
        sorted_spd.pop(i)
        total_fleet -= 1

    i -= 1

print(total_fleet)

7
